<a href="https://colab.research.google.com/github/iam-Dylan/automated-essay-scoring/blob/dylan/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google.colab import drive
drive.mount('/gdrive')

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import lightgbm as lgb
from flaml import AutoML
import nltk
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings('ignore')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [10]:
train = pd.read_csv('/gdrive/MyDrive/PTDLTM - Project /Notebook/learning-agency-lab-automated-essay-scoring-2/train.csv')
train

,essay_id,full_text,score
0,000d118,Many people have car where they live. The thin...,3
1,000fe60,I am a scientist at NASA that is discussing th...,3
2,001ab80,People always wish they had the same technolog...,4
3,001bdc0,"We all heard about Venus, the planet without a...",4
4,002ba53,"Dear, State Senator\n\nThis is a letter to arg...",3
...,...,...,...
17302,ffd378d,"the story "" The Challenge of Exploing Venus "" ...",2
17303,ffddf1f,Technology has changed a lot of ways that we l...,4
17304,fff016d,If you don't like sitting around all day than ...,2
17305,fffb49b,"In ""The Challenge of Exporing Venus,"" the auth...",1


In [19]:
test = pd.read_csv('/gdrive/MyDrive/PTDLTM - Project /Notebook/learning-agency-lab-automated-essay-scoring-2/test.csv')
test_essay_id = test['essay_id']

In [12]:
vectorizer = TfidfVectorizer(min_df=.05)
train_tfid = vectorizer.fit_transform(train['full_text'])
test_tfid = vectorizer.transform(test['full_text'])

train_y = np.array(train['score'])

aml = AutoML()

aml.fit(train_tfid, train_y, estimator_list=['lgbm'], task='classification', metric='macro_f1', time_budget=600)

best_config = aml.best_config

model = lgb.LGBMClassifier(**best_config)

model.fit(train_tfid, train_y)

[flaml.automl.logger: 05-23 09:47:28] {1680} INFO - task = classification
[flaml.automl.logger: 05-23 09:47:28] {1691} INFO - Evaluation method: holdout
[flaml.automl.logger: 05-23 09:47:28] {1789} INFO - Minimizing error metric: 1-macro_f1
[flaml.automl.logger: 05-23 09:47:28] {1901} INFO - List of ML learners in AutoML Run: ['lgbm']
[flaml.automl.logger: 05-23 09:47:28] {2219} INFO - iteration 0, current learner lgbm
[flaml.automl.logger: 05-23 09:47:29] {2345} INFO - Estimated sufficient time budget=13644s. Estimated necessary time budget=14s.
[flaml.automl.logger: 05-23 09:47:29] {2392} INFO -  at 1.4s,	estimator lgbm's best error=0.9072,	best estimator lgbm's best error=0.9072
[flaml.automl.logger: 05-23 09:47:29] {2219} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 05-23 09:47:34] {2392} INFO -  at 6.1s,	estimator lgbm's best error=0.9072,	best estimator lgbm's best error=0.9072
[flaml.automl.logger: 05-23 09:47:34] {2219} INFO - iteration 2, current learner lgbm

LGBMClassifier(colsample_bytree=0.7622839098563244,
               learning_rate=0.16075432906282125, log_max_bin=9,
               min_child_samples=3, n_estimators=142, num_leaves=16,
               reg_alpha=0.02046640007359354, reg_lambda=0.4809257969279843)

In [21]:
score = model.predict(test_tfid)

submission = pd.DataFrame({'essay_id': test_essay_id, 'score': score})
display(submission)

submission.to_csv('submission.csv')

[LightGBM] [Warning] Unknown parameter: log_max_bin


,essay_id,score
0,000d118,3
1,000fe60,3
2,001ab80,4


Submission generated successfully.
